## 6.1 CellChat

In [ ]:
library(CellChat)
library(patchwork)
library(dplyr)
library(Seurat)

In [ ]:
## Cellchat calculations are performed with deconvoluted data
## Key code are showen

In [ ]:
assignLabels <- function(object, prediction = "predictions") {
  pred <- object[[prediction]]@data
  pred <- pred[1:(nrow(pred)-1), ]
  # label each spot based on the maximum prediction probability
  labels = rownames(pred)[apply(pred, 2, which.max)]
  names(labels) <- colnames(pred)
  object$labels <- factor(labels)
  Idents(object) <- "labels"
  return(object)
}

Sol <- assignLabels(spe.Sol2, prediction = "predictions")
EDL <- assignLabels(spe.EDL2, prediction = "predictions")
LD <- assignLabels(spe.LD2, prediction = "predictions")

data.input1 = Seurat::GetAssayData(Sol, slot = "data", assay = "Spatial") 
data.input2 = Seurat::GetAssayData(EDL, slot = "data", assay = "Spatial") 
data.input3 = Seurat::GetAssayData(LD, slot = "data", assay = "Spatial") 

genes1 <- rownames(data.input1)
genes2 <- rownames(data.input2)
genes3 <- rownames(data.input3)

genes.common <- Reduce(intersect, list(genes1, genes2, genes3))

colnames(data.input1) <- paste0("A1_", colnames(data.input1))
colnames(data.input2) <- paste0("A2_", colnames(data.input2))
colnames(data.input3) <- paste0("A3_", colnames(data.input3))
data.input <- cbind(data.input1[genes.common, ], data.input2[genes.common, ], data.input3[genes.common, ])

# define the meta data
meta1 = data.frame(labels = Idents(Sol), samples = "A1") # manually create a dataframe consisting of the cell labels
meta2 = data.frame(labels = Idents(EDL), samples = "A2") 
meta3 = data.frame(labels = Idents(LD), samples = "A3") 

meta <- rbind(meta1, meta2, meta3)
rownames(meta) <- colnames(data.input)

meta$labels <- factor(meta$labels, levels = levels(Idents(Sol)))
meta$samples <- factor(meta$samples, levels = c("A1", "A2", "A3"))
unique(meta$labels) 
unique(meta$samples)

# load spatial transcriptomics information
Sol_positions <- read_csv("Sol_positions.csv")
LD_positions <- read_csv("LD_positions.csv")
EDL_positions <- read_csv("EDL_positions.csv")

Sol_positions <- as.data.frame(Sol_positions)
LD_positions <- as.data.frame(LD_positions)
EDL_positions <- as.data.frame(EDL_positions)

rownames(Sol_positions) <- Sol_positions[[1]]
Sol_positions <- Sol_positions[ , -1]

rownames(LD_positions) <- LD_positions[[1]]
LD_positions <- LD_positions[ , -1]

rownames(EDL_positions) <- EDL_positions[[1]]
EDL_positions <- EDL_positions[ , -1]

spatial.locs_Sol <- Sol_positions[, c("pxl_row_in_fullres", "pxl_col_in_fullres")]
spatial.locs_EDL <- EDL_positions[, c("pxl_row_in_fullres", "pxl_col_in_fullres")]
spatial.locs_LD <- LD_positions[, c("pxl_row_in_fullres", "pxl_col_in_fullres")]

rownames(spatial.locs_Sol) <- paste0("A1_", rownames(spatial.locs_Sol))
rownames(spatial.locs_EDL) <- paste0("A2_", rownames(spatial.locs_EDL))
rownames(spatial.locs_LD) <- paste0("A3_", rownames(spatial.locs_LD))

spatial.locs <- rbind(spatial.locs_Sol, spatial.locs_EDL, spatial.locs_LD)
rownames(spatial.locs) <- colnames(data.input)

matching_rows <- rownames(spatial.locs) %in% colnames(data.input)
cat("Number of matching rows: ", sum(matching_rows), "\n")
spatial.locs <- spatial.locs[matching_rows, ]
rownames(spatial.locs) <- colnames(data.input)[match(rownames(spatial.locs), colnames(data.input))]

scalefactors1 = jsonlite::fromJSON(txt = file.path("Z:/A/ST/Analysis/Results7/cellchatv2", 'Sol_scalefactors_json.json'))
scalefactors2 = jsonlite::fromJSON(txt = file.path("Z:/A/ST/Analysis/Results7/cellchatv2", 'EDL_scalefactors_json.json'))
scalefactors3 = jsonlite::fromJSON(txt = file.path("Z:/A/ST/Analysis/Results7/cellchatv2", 'LD_scalefactors_json.json'))
spot.size = 65

conversion.factor1 = spot.size/scalefactors1$spot_diameter_fullres
conversion.factor2 = spot.size/scalefactors2$spot_diameter_fullres
conversion.factor3 = spot.size/scalefactors3$spot_diameter_fullres

spatial.factors1 = data.frame(ratio = conversion.factor1, tol = spot.size/2)
spatial.factors2 = data.frame(ratio = conversion.factor2, tol = spot.size/2)
spatial.factors3 = data.frame(ratio = conversion.factor3, tol = spot.size/2)

spatial.factors <- rbind(spatial.factors1, spatial.factors2, spatial.factors3)
rownames(spatial.factors) <- c("A1", "A2", "A3")

cellchat <- createCellChat(object = data.input, meta = meta, group.by = "labels", datatype = "spatial", coordinates = spatial.locs, spatial.factors = spatial.factors)
## ...

## 6.2 Gene expression analysis

In [ ]:
#Fiugure 6H
plot_gene_expression <- function(seurat_object, genes, colors, border_size = 0.1, border_color = "black", 
                                 min_alpha = 0.1, r = 100) {
  expression_data <- FetchData(seurat_object, vars = genes)
  coords <- GetTissueCoordinates(seurat_object, image = "slice1")
  plot_data <- cbind(coords, expression_data)
  plot_data_long <- plot_data %>%
    pivot_longer(
      cols = all_of(genes),
      names_to = "gene",
      values_to = "expression"
    )
  p <- ggplot(plot_data_long) +
    geom_arc_bar(
      aes(
        x0 = x, y0 = y,
        r0 = 0, r = r, 
        start = case_when(
          gene == genes[1] ~ 0,
          gene == genes[2] ~ 2 * pi / 3,
          gene == genes[3] ~ 4 * pi / 3
        ),
        end = case_when(
          gene == genes[1] ~ 2 * pi / 3,
          gene == genes[2] ~ 4 * pi / 3,
          gene == genes[3] ~ 2 * pi
        ),
        fill = gene,  
        alpha = pmax(expression, min_alpha)  
      ),
      color = border_color, 
      size = border_size  
    ) +
    scale_fill_manual(
      values = colors 
    ) +
    scale_alpha_continuous(range = c(min_alpha, 1)) +  
    coord_fixed() +
    scale_y_reverse() + 
    theme_void() +
    theme(legend.position = "right") +
    labs(fill = "Gene", alpha = "Expression Level")
  print(p)
}

plot_gene_expression(
  seurat_object = EDL,
  genes = c("BMP7", "BMPR1A", "BMPR2"),
  colors = c("BMP7" = "red", "BMPR1A" = "green", "BMPR2" = "blue"),
  border_size = 0.2,
  border_color = "black", 
  min_alpha = 0.01,
  r = 150
)